In [36]:
from ensemble_boxes import *
from glob import glob
import pathlib
import os

In [37]:
mode = "modern"

In [38]:

if mode == "modern":
    yolov8_result_path = pathlib.Path("/workspace/runs/detect/predict")
    yolov5_result_path = pathlib.Path("/workspace/yolov5/runs/detect/exp")
elif mode == "old":
    yolov8_result_path = pathlib.Path("/workspace/runs/detect/predict2")
    yolov5_result_path = pathlib.Path("/workspace/yolov5/runs/detect/exp2")
image_id_list = [file.stem for file in list(yolov5_result_path.iterdir()) if file.is_file()]

yolov8_result_path

PosixPath('/workspace/runs/detect/predict')

In [39]:
def get_yolov5_label_path(id: str) -> pathlib.Path:
    return yolov5_result_path / "labels" / f"{id}.txt"

def get_yolov8_label_path(id: str) -> pathlib.Path:
    return yolov8_result_path / "labels" / f"{id}.txt"

def map_label_id_from_modern_to_all(modern_id: int, is_yolov8: bool) -> int:    
    if mode != "modern" or is_yolov8:
        return modern_id
    if modern_id == 0 or modern_id == 1:
        return modern_id
    return modern_id + 2

In [40]:
for v8file, v5file in zip(list(yolov8_result_path.iterdir()), list(yolov5_result_path.iterdir())):
    boxes_list = [[], []]

    scores_list = [[], []]
    labels_list = [[], []]
    weights = [1, 1]

    iou_thr = 0.5
    skip_box_thr = 0.001
    sigma = 0.1
    
    if not v5file.is_file() or not v8file.is_file():
        continue
    if v8file.stem != v5file.stem:
        continue
    name = v8file.stem    
    
    v5ann_file = get_yolov5_label_path(name)
    if not v5ann_file.exists():
        continue
    v5lines = v5ann_file.read_text().split("\n")
    for line in v5lines:
        if line == "":
            continue
        class_id, x_min, y_min, width, height, conf = map(float, line.split())
        class_id = int(class_id)
        class_id = map_label_id_from_modern_to_all(class_id, is_yolov8=False)
        boxes_list[0].append([x_min, y_min, x_min + width, y_min + height])
        labels_list[0].append(class_id)
        scores_list[0].append(conf)
    v8ann_file = get_yolov8_label_path(name)
    if not v8ann_file.exists():
        continue
    v8lines = v8ann_file.read_text().split("\n")
    for line in v8lines:
        if line == "":
            continue
        class_id, x_min, y_min, width, height, conf = map(float, line.split())
        class_id = map_label_id_from_modern_to_all(int(class_id), is_yolov8=True)
        boxes_list[1].append([x_min, y_min, x_min + width, y_min + height])
        labels_list[1].append(class_id)
        scores_list[1].append(conf)
        
    boxes, scores, labels = nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr)
    os.makedirs(f"/workspace/emsemble/{mode}-ret", exist_ok=True)
    out_path = pathlib.Path(f"/workspace/emsemble/{mode}-ret")
    out_path = out_path / f"{name}.txt"
    out_path.unlink(missing_ok=True)
    for box, score, label in zip(boxes, scores, labels):                
        with open(out_path, "a") as f:
            f.write(f"{int(label)} {box[0]} {box[1]} {box[2] - box[0]} {box[3] - box[1]}") 
            f.write("\n")
    

Warning. Fixed 4 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 4 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 3 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 4 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 20 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 6 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 4 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 4 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 4 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 28 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 4 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 2 boxes coordinates > 1. C

In [41]:
v5file, labels, boxes

(PosixPath('/workspace/yolov5/runs/detect/exp/test_995415_0003.jpg'),
 array([1, 1, 4, 4, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 9, 9]),
 array([[0.490625  , 0.388125  , 1.        , 1.        ],
        [0.736207  , 0.435522  , 1.        , 0.919333  ],
        [0.351562  , 0.420625  , 0.373437  , 0.454375  ],
        [0.285625  , 0.368125  , 0.306875  , 0.428

In [42]:
import numpy as np
import pandas as pd

df = pd.DataFrame(scores_list)
df

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,162
0,0.075192,0.075263,0.075707,0.077963,0.078808,0.078970,0.079049,0.079418,0.079792,0.080732,...,0.306441,0.310233,0.311287,0.313367,0.317782,0.321032,0.322551,0.335178,0.337896,0.358729
1,0.129684,0.130299,0.131205,0.132470,0.132934,0.135018,0.135240,0.139360,0.141147,0.142794,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
